In [1]:
!pip install simpletransformers
!pip install spacy==3.2.1
!python -m spacy download pt_core_news_lg
!pip install seqval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 249 kB 35.6 MB/s 
     |████████████████████████████████| 1.8 MB 73.4 MB/s 
     |████████████████████████████████| 365 kB 69.8 MB/s 
     |████████████████████████████████| 6.6 MB 44.9 MB/s 
     |████████████████████████████████| 4.7 MB 47.6 MB/s 
     |████████████████████████████████| 43 kB 2.2 MB/s 
     |████████████████████████████████| 1.3 MB 60.0 MB/s 
     |████████████████████████████████| 9.1 MB 44.7 MB/s 
     |████████████████████████████████| 120 kB 55.9 MB/s 
     |████████████████████████████████| 181 kB 34.3 MB/s 
     |████████████████████████████████| 158 kB 75.2 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
     |████████████████████████████████| 157 kB 70.0 MB/s 
     |████████████████████████████████| 157 kB 68.8 MB/s 
     |████████████████████████████████| 157 kB 71.1 MB/s 
     |█████████████████████████████

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 577.4 MB 6.0 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_lg')
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement seqval (from versions: none)
ERROR: No matching distribution found for seqval


In [2]:
import requests
from tqdm import tqdm
from zipfile import ZipFile
import os 
import shutil

In [3]:
BASE_MODEL_DIR = './drive/Shareddrives/MEC - Correção textual/PLN/Notebooks/Pontuação/models'
MODEL_NAME = 'bert-portuguese-tedtalk2012'
zipfile = ZipFile(os.path.join(BASE_MODEL_DIR, f'{MODEL_NAME}.zip'))
zipfile.extractall(path=MODEL_NAME)

In [4]:
from simpletransformers.ner import NERModel, NERArgs
import torch

def get_model(model_path,
              model_type="bert",
              labels=None,
              max_seq_length=512):
    model_args = NERArgs()

    if labels is not None:
        model_args.labels_list = labels
    else:
        model_args.labels_list = ["O", "COMMA", "PERIOD", "QUESTION"]
    model_args.silent = True
    model_args.max_seq_length = max_seq_length
    return NERModel(
        model_type,
        model_path,
        args=model_args,
        use_cuda=torch.cuda.is_available()
    )

In [5]:
import json 
filename = '/content/drive/Shareddrives/MEC - Correção textual/PLN/Notebooks/Pontuação/Dataset/student.json'
student_entities = json.load(open(filename))


FileNotFoundError: ignored

In [ ]:
import nltk 
nltk.download('punkt')

In [ ]:
from nltk.tokenize import word_tokenize, wordpunct_tokenize

wordpunct_tokenize("Ela vai dormir.")

In [ ]:
import re
def split_paragraphs(text):
    paragraphs = text.split('\n')
    return paragraphs

In [ ]:
import string
def remove_punctuation(text):
    text = ' '.join(word for word in wordpunct_tokenize(text)
                    if word not in string.punctuation)
    return text

In [ ]:
def merge_dicts(dict_args):
    """
    Given any number of dictionaries, shallow copy and merge into a new dict,
    precedence goes to key-value pairs in latter dictionaries.
    """
    result = dict_args[0]
    for dictionary in dict_args:
        result.update(dictionary)
    return result

In [ ]:
import traceback
def get_labels(text, pred_dict):
    labels = []
    try:
        ## Tokenização do BERT tá diferente daque é feita aqui
        for word in wordpunct_tokenize(text):
            if word not in string.punctuation:
                if pred_dict[word] == "QUESTION":
                    label = "I-PERIOD"
                elif pred_dict[word] == "COMMA":
                    label = "I-COMMA"
                elif pred_dict[word] == "PERIOD":
                    label = "I-PERIOD"
                else:
                    label = "O"
                labels.append(label)
    except KeyError:
        print("KeyError", pred_dict)
        print(traceback.format_exc())
        print(text)
        
    return labels

In [ ]:
def preprocess_text(text):
    """ 
    Preprocess text for prediction
    :param text: text to preprocess
    :return:  list of preprocessed text
    """
    paragraphs = truncate_sentences(text)

    return list(map(lambda x: remove_punctuation(x), paragraphs))

In [ ]:
from itertools import chain
from transformers import BertTokenizer
MODEL_PATH = "./bert-portuguese-tedtalk2012"

bert_tokenizer = BertTokenizer.from_pretrained(MODEL_PATH)

In [ ]:
def truncate_sentences(text, max_seq_length=512, overlap=20):
    texts = []
    tokens = bert_tokenizer.tokenize(text)

    if len(tokens) > max_seq_length:
        if len(tokens) % max_seq_length != 0:
            max_seq_length //= 2

        for i in range(0, len(tokens), max_seq_length):
            slide = 0 if i == 0 else overlap
            truncated_tokens = tokens[i - slide:i + max_seq_length]
            new_text = bert_tokenizer.convert_tokens_to_string(truncated_tokens)

            texts.append(new_text)

        if len(bert_tokenizer.tokenize(texts[-1])) + len(bert_tokenizer.tokenize(texts[-2])) < max_seq_length:
            texts[-2] = texts[-2] + texts[-1]
            texts.pop()

    else:
        texts.append(text)
    return texts

In [ ]:
def predict(test_text: str, model):
    texts = preprocess_text(test_text)
    prediction_list, raw_outputs = model.predict(texts)
    pred_dict = merge_dicts(list(chain(*prediction_list)))

    return get_labels(test_text, pred_dict)

In [ ]:
def text2labels(sentence):
    """
    Convert text to labels
    :param sentence: text to convert
    :return:  list of labels
    """
    tokens = wordpunct_tokenize(sentence.lower())

    labels = []
    for i, token in enumerate(tokens):
        try:
            if token not in string.punctuation:
                labels.append('O')
            elif token in ['.', '?', '!', ';']:
                labels[-1] = 'I-PERIOD'
            elif token == ',':
                labels[-1] = 'I-COMMA'

        except IndexError:
            raise ValueError(f"Sentence can't start with punctuation {token}")
    return labels

In [ ]:
anno_file2 = "/content/drive/Shareddrives/MEC - Correção textual/PLN/Notebooks/Pontuação/Dataset/annotator2.json"
annotator2 = json.load(open(anno_file2, "r"))
MODEL_PATH = "./bert-portuguese-tedtalk2012"
model = get_model(MODEL_PATH, model_type="bert", max_seq_length=512)
annotator2[:1][:1]

In [ ]:

anno_files1 = '/content/drive/Shareddrives/MEC - Correção textual/PLN/Notebooks/Pontuação/Dataset/annotator1.json'
annotator1 = json.load(open(anno_files1))
annotator1[0]['labels'][:3]

In [ ]:
from seqeval import metrics
import pandas as pd
from tqdm import tqdm
bert_labels = []
true_labels = []

for item in tqdm(annotator2, total=len(annotator2)):

    text_id = item["text_id"]

    ann_text = item["text"]

    bert_label = predict(ann_text, model)
    true_label = text2labels(item["text"])
   
    true_labels.append(true_label)
    bert_labels.append(bert_label)

report = metrics.classification_report(true_labels, bert_labels, output_dict=True)
df = pd.DataFrame.from_dict(report, orient='index')
df

In [ ]:
df.to_csv(f"results_anotador2.csv")